In [1]:
import polars as pl
import numpy as np

In [2]:
df = pl.scan_parquet("data/combined_PISA_00_18_all.parquet")

In [3]:
df.columns

['index',
 'year',
 'country',
 'school_id',
 'student_id',
 'mother_educ',
 'father_educ',
 'gender',
 'computer',
 'internet',
 'math',
 'read',
 'science',
 'stu_wgt',
 'desk',
 'room',
 'dishwasher',
 'television',
 'computer_n',
 'car',
 'book',
 'wealth',
 'escs']

In [4]:
q2 = df.lazy().group_by(pl.col("year")).agg(pl.quantile("math", 0.99).alias("math99"), pl.quantile("read", 0.99).alias("read99"), pl.quantile("science", 0.99).alias("science99")).sort("year")
# this computation is flawed - should use weighted quantile

In [5]:
q = df.lazy().join(q2, on="year").group_by(pl.col("year"), pl.col("country")).agg(pl.mean("math"), pl.mean("read"), pl.mean("science"), (((pl.col("math") > pl.col("math99")) * 100 * pl.col("stu_wgt")).sum() / pl.col("stu_wgt").sum()).alias("in_math99"))

In [6]:
worldbank = pl.scan_parquet("data/worldbank_indicators.parquet").with_columns(pl.col("Time").cast(pl.Int64))

In [7]:
merged = q.join(worldbank, left_on=["country", "year"], right_on=["Country Code", "Time"], how="left")

In [8]:
print(merged.explain(optimized=True))

LEFT JOIN:
LEFT PLAN ON: [col("country"), col("year")]
  AGGREGATE
  	[col("math").mean(), col("read").mean(), col("science").mean(), [([([([(col("math")) > (col("math99"))].cast(Int32)) * (100)].cast(Float64)) * (col("stu_wgt"))].sum()) / (col("stu_wgt").sum())].alias("in_math99")] BY [col("year"), col("country")] FROM
    FAST_PROJECT: [year, math, read, science, math99, stu_wgt, country]
      INNER JOIN:
      LEFT PLAN ON: [col("year")]
        FAST_PROJECT: [year, math, read, science, stu_wgt, country]
          CACHE[id: aa26a40801c8cd74, count: 1]

              Parquet SCAN data/combined_PISA_00_18_all.parquet
              PROJECT 6/23 COLUMNS
      RIGHT PLAN ON: [col("year")]
        SORT BY [col("year")]
          AGGREGATE
          	[col("math").quantile().alias("math99"), col("read").quantile().alias("read99"), col("science").quantile().alias("science99")] BY [col("year")] FROM
            FAST_PROJECT: [year, math, read, science]
              CACHE[id: aa26a40801c8cd7

In [9]:
df2 = merged.collect()

In [10]:
df2.filter(pl.col("country") == "USA").sort("year")

year,country,math,read,science,in_math99,gdp_pc,gdp_pc_growth,primary_completion,lower_sec_completion,upper_sec_completion,population
i64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,u64
2000,"""USA""",483.273001,495.653837,490.537116,0.321525,36329.956073,2.925441,null,null,null,282162411
2003,"""USA""",481.961563,493.767805,490.007714,0.565479,39490.274956,1.91648,null,null,null,290107933
2006,"""USA""",475.177462,null,488.291876,0.486487,46302.00088,1.796486,98.54126,94.191933,85.751007,298379912
2009,"""USA""",485.637508,497.591875,499.626656,0.340298,47194.943355,-3.450016,98.594177,94.535217,86.709084,306771529
2012,"""USA""",481.032476,497.471639,497.788388,0.401697,51784.418574,1.533102,98.790482,94.984489,87.6791,313877662
2018,"""USA""",473.142717,500.15018,497.277201,0.427504,62823.309438,2.404868,99.031532,96.029373,89.805359,326838199


In [11]:
df2.filter(pl.col("country").str.contains("CAN")).sort("year")

year,country,math,read,science,in_math99,gdp_pc,gdp_pc_growth,primary_completion,lower_sec_completion,upper_sec_completion,population
i64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,u64
2000,"""CAN""",522.717763,523.567877,520.791287,0.704775,24271.002056,4.163949,null,null,null,30685730
2003,"""CAN""",521.631895,516.088253,508.497317,1.497084,28300.463096,0.892853,null,null,null,31644028
2006,"""CAN""",517.446121,512.42873,522.5038,1.636177,40504.060725,1.606178,null,null,79.403389,32571174
2009,"""CAN""",516.392387,511.531936,518.117041,1.597291,40876.310154,-4.017257,null,null,null,33628895
2012,"""CAN""",509.328982,511.235906,514.712929,1.348539,52669.089963,0.656758,null,null,null,34714222
2018,"""CAN""",503.450271,509.466476,509.894194,2.039556,46548.638411,1.301965,null,null,null,37065084


In [12]:
df2.shape

(360, 12)

In [13]:
df2.head()

year,country,math,read,science,in_math99,gdp_pc,gdp_pc_growth,primary_completion,lower_sec_completion,upper_sec_completion,population
i64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,u64
2000,"""BGR""",432.166333,424.442482,442.851508,0.337287,1621.262107,5.105057,null,null,null,8170172
2000,"""CHE""",529.359372,497.897374,497.673435,0.970659,38865.02194,3.376157,null,null,null,7184250
2000,"""IRL""",499.922874,525.052807,514.282774,0.137622,26334.567205,7.954806,null,null,null,3805174
2000,"""PRT""",455.83801,476.354926,464.356791,0.029423,11526.372067,3.089054,78.558182,25.260309,15.77243,10289898
2000,"""THA""",444.679506,435.85854,444.772084,0.024493,2004.107059,3.421815,null,null,null,63066603


In [14]:
import statsmodels.formula.api as smf
import pandas as pd

In [15]:
pd_df = df2.to_pandas()

In [55]:
df2.describe()

statistic,year,country,math,read,science,in_math99,gdp_pc,gdp_pc_growth,primary_completion,lower_sec_completion,upper_sec_completion,population
str,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""",360.0,"""360""",359.0,357.0,359.0,360.0,343.0,342.0,126.0,144.0,158.0,343.0
"""null_count""",0.0,"""0""",1.0,3.0,1.0,0.0,17.0,18.0,234.0,216.0,202.0,17.0
"""mean""",2009.308333,null,469.566848,465.64966,471.923403,0.984724,27157.540836,1.655096,89.066682,74.660638,60.460062,3.6713e7
"""std""",5.901413,null,57.846265,52.065152,52.738004,1.771274,24604.229288,4.392687,11.072305,21.041142,21.202768,5.9855e7
"""min""",2000.0,"""ALB""",315.963154,290.918937,326.40745,0.0,543.110702,-17.145394,51.353222,25.260309,15.77243,33026.0
"""25%""",2006.0,null,426.894651,425.347082,429.386717,0.066259,8003.814493,-0.185773,81.543251,57.16,41.57,4.564855e6
"""50%""",2009.0,null,486.185453,480.551538,487.156522,0.471216,20137.591222,1.92983,92.483467,79.658386,64.448914,1.0193798e7
"""75%""",2012.0,null,511.663285,502.90935,510.583331,1.118103,40870.852365,3.953911,98.790482,94.448936,79.004883,4.4622518e7
"""max""",2018.0,"""YUG""",610.54703,567.419714,593.636845,16.089708,149461.785571,33.030488,100.0,99.816658,97.399788,3.26838199e8


In [59]:
pd_df[["math", "read", "science", "in_math99", "gdp_pc", "gdp_pc_growth"]].corr()

,math,read,science,in_math99,gdp_pc,gdp_pc_growth
math,1.000000,0.942113,0.966018,0.596006,0.501018,-0.029224
read,0.942113,1.000000,0.970027,0.509521,0.478101,-0.119945
science,0.966018,0.970027,1.000000,0.540994,0.464683,-0.097748
in_math99,0.596006,0.509521,0.540994,1.000000,0.392876,-0.060483
gdp_pc,0.501018,0.478101,0.464683,0.392876,1.000000,-0.196734
gdp_pc_growth,-0.029224,-0.119945,-0.097748,-0.060483,-0.196734,1.000000


In [16]:
import plotly.express as px

In [65]:
smf.ols("gdp_pc_growth ~ gdp_pc + math + read + science + in_math99 + population + C(year)", pd_df).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          gdp_pc_growth   R-squared:                       0.542
Model:                            OLS   Adj. R-squared:                  0.526
Method:                 Least Squares   F-statistic:                     35.15
Date:                Tue, 26 Mar 2024   Prob (F-statistic):           4.69e-49
Time:                        14:24:36   Log-Likelihood:                -850.83
No. Observations:                 339   AIC:                             1726.
Df Residuals:                     327   BIC:                             1772.
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           9.2274      2.127      4.339      0.000       5.043      13.411
C(year)[T.2003]    -1.5882      0.685     -2.318      0.021      -2.936      -0.240
C(year)[T.2006]     1.2696      0.657      1.934      0.054      -0.022       2.561
C(year)[T.2009]    -7.2723      0.625    -11.635      0.000      -8.502      -6.043
C(year)[T.2012]    -2.1079      0.636     -3.312      0.001      -3.360      -0.856
C(year)[T.2018]    -1.5439      0.618     -2.498      0.013      -2.760      -0.328
gdp_pc          -3.685e-05   8.38e-06     -4.397      0.000   -5.33e-05   -2.04e-05
math                0.0809      0.013      6.345      0.000       0.056       0.106
read               -0.0315      0.014     -2.291      0.023      -0.059      -0.004
science            -0.0589      0.018     -3.354      0.001      -0.093      -0.024
in_math99           0.0011      0.189      0.006      0.995      -0.371       0.373
population       2.605e-09   2.95e-09      0.882      0.378    -3.2e-09    8.41e-09
==============================================================================
Omnibus:                       53.831   Durbin-Watson:                   2.044
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              355.973
Skew:                           0.407   Prob(JB):                     5.03e-78
Kurtosis:                       7.954   Cond. No.                     9.13e+08
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 9.13e+08. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [68]:
fig = px.scatter(df2, y="in_math99", x="gdp_pc_growth", color='gdp_pc', hover_data=["country", "year"])
fig.show()

In [26]:
fig = px.scatter(df2, y="math", x="gdp_pc_growth", color='gdp_pc', hover_data="country")
fig.show()

In [20]:
fig2 = px.line(df2.filter(pl.col("country").str.contains_any(["USA", "SGP", "CAN", "GBP", "JPN", "KOR", "MYS", "FIN"])).sort("year"), x="year", y="in_math99", color="country")
fig2.show()

In [21]:
fig2 = px.line(df2.filter(pl.col("country").str.contains_any(["USA", "SGP", "CAN", "GBP", "JPN", "KOR", "MYS", "FIN"])).sort("year"), x="year", y="math", color="country")
fig2.show()